<a href="https://colab.research.google.com/github/krakowiakpawel9/neural-network-course/blob/master/03_keras/03_overfitting_underfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Główne problemy uczenia maszynowego: przeuczenie (overfitting) oraz niedouczenie (underfitting)

>Celem tego notebook'a jest pokazanie przykładów zbyt dobrego dopasowanie modelu do danych uczących (przeuczenie) oraz zbyt słabego dopasowania modelu do danych uczących (niedouczenie).
>
>Wykorzystamy zbiór z bilioteki Keras składający się z 25000 recenzji filmów oznaczonych sentymentem: pozytywna/negatywna. Recenzje są wstępnie przetworzone, a każda recenzja jest zakodowana jako sekwencja indeksów słów. Słowa są indeksowane według ogólnej częstotliwości w zbiorze danych. Na przykład liczba 5 oznacza piąte najczęściej pojawiające się słowo w danych. Liczba 0 nie oznacza określonego słowa.

### Spis treści
1. [Import bibliotek](#a1)
2. [Załadowanie i przygotowanie danych](#a2)
3. [Budowa modelu bazowego](#a3)
4. [Budowa 'mniejszego' modelu](#a4)    
5. [Budowa 'większego' modelu](#a5)
6. [Porównanie wydajności modeli](#a6)
7. [Metody regularyzacji](#a7)

### <a name='a1'></a> 1. Import bibliotek

In [2]:
%tensorflow_version 2.x
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__

TensorFlow 2.x selected.


'2.0.0'

### <a name='a2'></a> 2. Załadowanie i przygotowanie danych

In [0]:
NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

In [4]:
print(f'train_data shape: {train_data.shape}')
print(f'test_data shape: {test_data.shape}')

train_data shape: (25000,)
test_data shape: (25000,)


In [5]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [6]:
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))


<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [7]:
train_labels[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [8]:
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

(10000,)

In [16]:
test_data.shape

(25000, 10000)

### <a name='a3'></a> 3. Budowa modelu bazowego

In [9]:
baseline_model = Sequential()
baseline_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
baseline_model.add(Dense(16, activation='relu'))
baseline_model.add(Dense(1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                160016    
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [10]:
baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 5s 188us/sample - loss: 0.5168 - accuracy: 0.7975 - binary_crossentropy: 0.5168 - val_loss: 0.3760 - val_accuracy: 0.8672 - val_binary_crossentropy: 0.3760
Epoch 2/20
25000/25000 [==============================] - 4s 141us/sample - loss: 0.2794 - accuracy: 0.9030 - binary_crossentropy: 0.2794 - val_loss: 0.2922 - val_accuracy: 0.8852 - val_binary_crossentropy: 0.2922
Epoch 3/20
25000/25000 [==============================] - 4s 142us/sample - loss: 0.2022 - accuracy: 0.9299 - binary_crossentropy: 0.2022 - val_loss: 0.2829 - val_accuracy: 0.8878 - val_binary_crossentropy: 0.2829
Epoch 4/20
25000/25000 [==============================] - 4s 142us/sample - loss: 0.1617 - accuracy: 0.9447 - binary_crossentropy: 0.1617 - val_loss: 0.2997 - val_accuracy: 0.8819 - val_binary_crossentropy: 0.2997
Epoch 5/20
25000/25000 [==============================] - 4s 143us/sample - loss: 0.1350 - acc

### <a name='a4'></a> 3. Budowa 'mniejszego' modelu

In [11]:
smaller_model = Sequential()
smaller_model.add(Dense(4, activation='relu', input_shape=(NUM_WORDS,)))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 4)                 40004     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 40,029
Trainable params: 40,029
Non-trainable params: 0
_________________________________________________________________


In [12]:
smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 4s 161us/sample - loss: 0.5441 - accuracy: 0.7834 - binary_crossentropy: 0.5441 - val_loss: 0.4263 - val_accuracy: 0.8594 - val_binary_crossentropy: 0.4263
Epoch 2/20
25000/25000 [==============================] - 3s 135us/sample - loss: 0.3357 - accuracy: 0.8930 - binary_crossentropy: 0.3357 - val_loss: 0.3309 - val_accuracy: 0.8789 - val_binary_crossentropy: 0.3309
Epoch 3/20
25000/25000 [==============================] - 3s 137us/sample - loss: 0.2542 - accuracy: 0.9176 - binary_crossentropy: 0.2542 - val_loss: 0.2976 - val_accuracy: 0.8858 - val_binary_crossentropy: 0.2976
Epoch 4/20
25000/25000 [==============================] - 3s 136us/sample - loss: 0.2107 - accuracy: 0.9316 - binary_crossentropy: 0.2107 - val_loss: 0.2871 - val_accuracy: 0.8873 - val_binary_crossentropy: 0.2871
Epoch 5/20
25000/25000 [==============================] - 3s 138us/sample - loss: 0.1821 - acc

### <a name='a5'></a> 4. Budowa 'większego' modelu

In [13]:
bigger_model = Sequential()
bigger_model.add(Dense(512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               5120512   
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 513       
Total params: 5,383,681
Trainable params: 5,383,681
Non-trainable params: 0
_________________________________________________________________


In [14]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 14s 561us/sample - loss: 0.3531 - accuracy: 0.8466 - binary_crossentropy: 0.3531 - val_loss: 0.3099 - val_accuracy: 0.8741 - val_binary_crossentropy: 0.3099
Epoch 2/20
25000/25000 [==============================] - 13s 538us/sample - loss: 0.1458 - accuracy: 0.9482 - binary_crossentropy: 0.1458 - val_loss: 0.3265 - val_accuracy: 0.8759 - val_binary_crossentropy: 0.3265
Epoch 3/20
25000/25000 [==============================] - 14s 543us/sample - loss: 0.0501 - accuracy: 0.9856 - binary_crossentropy: 0.0501 - val_loss: 0.4404 - val_accuracy: 0.8702 - val_binary_crossentropy: 0.4404
Epoch 4/20
25000/25000 [==============================] - 14s 542us/sample - loss: 0.0071 - accuracy: 0.9990 - binary_crossentropy: 0.0071 - val_loss: 0.5463 - val_accuracy: 0.8687 - val_binary_crossentropy: 0.5463
Epoch 5/20
25000/25000 [==============================] - 13s 538us/sample - loss: 0.0016 

In [15]:
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.516810,0.79748,0.516810,0.376013,0.86716,0.376013,0
1,0.279368,0.90296,0.279368,0.292208,0.88520,0.292208,1
2,0.202157,0.92992,0.202157,0.282876,0.88780,0.282876,2
3,0.161734,0.94468,0.161734,0.299662,0.88192,0.299662,3
4,0.134971,0.95544,0.134971,0.317094,0.87932,0.317094,4


### <a name='a6'></a> 5. Porównanie wydajności modeli

In [16]:
for history in [smaller_history, baseline_history, bigger_history]:
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
hist.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.353076,0.84660,0.353076,0.309949,0.87412,0.309949,0
1,0.145822,0.94816,0.145822,0.326519,0.87592,0.326519,1
2,0.050107,0.98560,0.050107,0.440353,0.87020,0.440353,2
3,0.007124,0.99896,0.007124,0.546278,0.86868,0.546278,3
4,0.001591,0.99996,0.001591,0.637632,0.87148,0.637632,4


In [17]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

### <a name='a7'></a> 6. Metody regularyzacji

In [18]:
from tensorflow.keras.regularizers import l2

l2_model = Sequential()
l2_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [19]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 4s 166us/sample - loss: 0.6840 - accuracy: 0.8068 - binary_crossentropy: 0.4955 - val_loss: 0.5381 - val_accuracy: 0.8721 - val_binary_crossentropy: 0.3638
Epoch 2/20
25000/25000 [==============================] - 3s 138us/sample - loss: 0.4476 - accuracy: 0.9042 - binary_crossentropy: 0.2806 - val_loss: 0.4523 - val_accuracy: 0.8870 - val_binary_crossentropy: 0.2940
Epoch 3/20
25000/25000 [==============================] - 4s 142us/sample - loss: 0.3738 - accuracy: 0.9223 - binary_crossentropy: 0.2232 - val_loss: 0.4256 - val_accuracy: 0.8874 - val_binary_crossentropy: 0.2835
Epoch 4/20
25000/25000 [==============================] - 4s 140us/sample - loss: 0.3336 - accuracy: 0.9325 - binary_crossentropy: 0.1979 - val_loss: 0.4123 - val_accuracy: 0.8846 - val_binary_crossentropy: 0.2835
Epoch 5/20
25000/25000 [==============================] - 3s 139us/sample - loss: 0.3079 - acc

In [20]:
fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [21]:
from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [22]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 4s 174us/sample - loss: 0.6517 - accuracy: 0.6289 - binary_crossentropy: 0.6517 - val_loss: 0.5857 - val_accuracy: 0.7598 - val_binary_crossentropy: 0.5857
Epoch 2/20
25000/25000 [==============================] - 4s 141us/sample - loss: 0.5470 - accuracy: 0.8072 - binary_crossentropy: 0.5470 - val_loss: 0.4959 - val_accuracy: 0.8427 - val_binary_crossentropy: 0.4959
Epoch 3/20
25000/25000 [==============================] - 4s 143us/sample - loss: 0.4770 - accuracy: 0.8700 - binary_crossentropy: 0.4770 - val_loss: 0.4476 - val_accuracy: 0.8663 - val_binary_crossentropy: 0.4476
Epoch 4/20
25000/25000 [==============================] - 4s 140us/sample - loss: 0.4183 - accuracy: 0.8972 - binary_crossentropy: 0.4183 - val_loss: 0.4124 - val_accuracy: 0.8834 - val_binary_crossentropy: 0.4124
Epoch 5/20
25000/25000 [==============================] - 4s 140us/sample - loss: 0.3696 - acc

In [23]:
fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()